In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Plotting
import plotly.express as px
import matplotlib.pyplot as plt
import streamlit as st

import datetime

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import quant finance libraries
import yfinance as yf

In [10]:
stocks = ['tsla']
start_date = datetime.date(2020, 1, 1)

In [11]:
def get_data(stocks, start_date, end_date):
            return yf.download(stocks, start = start_date, end = end_date)

In [12]:
stocks

['tsla']

In [13]:
start_date

datetime.date(2020, 1, 1)

In [33]:
df = get_data(stocks, start_date = "2019-01-01", end_date = "2022-11-30")       
#df = df['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [28]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-31,22.519333,22.614000,21.684000,22.186666,22.186666,94534500
2019-01-02,20.406668,21.008667,19.920000,20.674667,20.674667,174879000
2019-01-03,20.466667,20.626667,19.825333,20.024000,20.024000,104478000
2019-01-04,20.400000,21.200001,20.181999,21.179333,21.179333,110911500
2019-01-07,21.448000,22.449333,21.183332,22.330667,22.330667,113268000


In [70]:
def compute_strat_8(df, capital):
    df["buy"] = 0
    df["sell"] = 0
    df["position"] = 0

    # Define Fibonacci levels
    fibonacci_levels = [0.236, 0.382, 0.618]

    for i in range(2, len(df)):
        current_price = df.iloc[i-2]["Close"]
        previous_high = df.iloc[-4:i-2]["High"].max()
        previous_low = df.iloc[-4:i-2]["Low"].min()

        price_range = previous_high - previous_low

        buy_price = current_price - (fibonacci_levels[1] * price_range)
        sell_price = current_price + (fibonacci_levels[1] * price_range)

        if df.iloc[i - 1]["position"] == 1:
            if current_price >= sell_price:
                capital += df.iloc[i - 1]["position"] * current_price
                df.at[i, "sell"] = 1
                df.at[i, "position"] = 0
        else:
            if current_price <= buy_price:
                shares_to_buy = int(capital / current_price)
                capital -= shares_to_buy * current_price
                df.at[i, "buy"] = 1
                df.at[i, "position"] = shares_to_buy

    return df


In [55]:
current_price = df.iloc[len(df)-2]["Close"]
previous_high = df.iloc[-5:len(df)-2]["High"].max()
previous_low = df.iloc[-5:len(df)-2]["Low"].min()

In [51]:
fibonacci_levels = [0.236, 0.382, 0.618]

In [65]:
price_range = previous_high - previous_low
buy_price = current_price - (fibonacci_levels[1] * price_range)
sell_price = current_price + (fibonacci_levels[1] * price_range)

In [57]:
current_price, previous_high, previous_low

(182.9199981689453, 185.1999969482422, 166.19000244140625)

In [66]:
price_range, buy_price, sell_price

(19.009994506835938, 175.658180267334, 190.18181607055664)

In [71]:
df_results = compute_strat_8(df, 1000)

In [72]:
df_results.tail(12)

,Open,High,Low,Close,Adj Close,Volume,buy,sell,position
Date,,,,,,,,,
2022-11-11,186.000000,196.520004,182.589996,195.970001,195.970001,114403600,0,0,0
2022-11-14,192.770004,195.729996,186.339996,190.949997,190.949997,92226600,0,0,0
2022-11-15,195.880005,200.820007,192.059998,194.419998,194.419998,91293800,0,0,0
2022-11-16,191.509995,192.570007,185.660004,186.919998,186.919998,66567600,0,0,0
2022-11-17,183.960007,186.160004,180.899994,183.169998,183.169998,64336000,0,0,0
2022-11-18,185.050003,185.190002,176.550003,180.190002,180.190002,76048900,0,0,0
2022-11-21,175.850006,176.770004,167.539993,167.869995,167.869995,92882700,0,0,0
2022-11-22,168.630005,170.919998,166.190002,169.910004,169.910004,78452300,0,0,0
2022-11-23,173.570007,183.619995,172.500000,183.199997,183.199997,109536700,0,0,0


In [20]:
final_capital

1188.813309188609

In [7]:
def cum_returns(stocks, wts):

  weighted_returns = (wts * stocks.pct_change()[1:])
  weighted_returns = pd.DataFrame(weighted_returns)
  port_ret = weighted_returns.sum(axis=1)
  return (port_ret + 1).cumprod() 

In [8]:
total_stocks = len(stocks)
weight = [1/total_stocks]*total_stocks
#call cumulative returns
returns = cum_returns(df, weight).reset_index()
returns.rename(columns={'Date':'date', 0:'ret'}, inplace=True)

In [9]:
# optimizitation
from scipy.optimize import minimize 

def optimize_weights(returns, risk_free_return):
    
    n = returns.shape[1]
    initial_weights = np.ones(n) / n
    bounds = [(0, 1) for i in range(n)]
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    def neg_sharpe_ratio(weights, returns, risk_free_rate):
        portfolio_return = np.sum(returns.mean() * weights) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        return -sharpe_ratio
    result = minimize(fun=neg_sharpe_ratio, x0=initial_weights, args=(returns, risk_free_return), method='SLSQP', bounds=bounds, constraints=constraints)
    optimized_weights = result.x
    return optimized_weights

In [10]:
res = optimize_weights(df.pct_change(), 4)

In [11]:
pd.DataFrame(data=res, index=df.columns, columns=['res']).sort_values(by='res', ascending=False)[:3].index.tolist()

['NVDA', 'SAN.PA', 'ENG.MC']

In [15]:
returns = df.pct_change()
n = returns.shape[1]
weights = np.ones(n) / n
np.sum(returns.mean() * weights) * 252

0.17876103188362516

In [12]:
import tensorflow as tf

def sharpe_ratio_loss(y_true, y_pred):
    # Sharpe ratio formula
    return -tf.reduce_mean((y_true - y_pred) / tf.math.reduce_std(y_true - y_pred))

# Define the inputs
x = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
y = tf.constant([0.5, 1.0, 1.5, 2.0, 2.5])

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss=sharpe_ratio_loss)

# Train the model
history = model.fit(x, y, epochs=1000, verbose=0)

# Get the gradients
with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = sharpe_ratio_loss(y, y_pred)
grads = tape.gradient(loss, model.trainable_variables)

print(grads)

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00019751]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00481659], dtype=float32)>]
